In [1]:
"""
Tutorial for the Million Song Dataset

by Thierry Bertin-Mahieux (2011) Columbia University
   tb2332@columbia.edu
   Copyright 2011 T. Bertin-Mahieux, All Rights Reserved

This tutorial will walk you through a quick experiment
using the Million Song Dataset (MSD). We will actually be working
on the 10K songs subset for speed issues, but the code should
transpose seamlessly.

In this tutorial, we do simple metadata analysis. We look at
which artist has the most songs by iterating over the whole
dataset and using an SQLite database.

You need to have the MSD code downloaded from GITHUB.
See the MSD website for details:
http://labrosa.ee.columbia.edu/millionsong/

If you have any questions regarding the dataset or this tutorial,
please first take a look at the website. Send us an email
if you haven't found the answer.

Note: this tutorial is developed using Python 2.6
      on an Ubuntu machine. PDF created using 'pyreport'.
"""

# usual imports
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np # get it at: http://numpy.scipy.org/
# path to the Million Song Dataset subset (uncompressed)
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_subset_path='./data/MSD/MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check
# path to the Million Song Dataset code
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_code_path='./'
assert os.path.isdir(msd_code_path),'wrong path' # sanity check
# we add some paths to python so we can import MSD code
# Ubuntu: you can change the environment variable PYTHONPATH
# in your .bashrc file so you do not have to type these lines
sys.path.append(os.path.join(msd_code_path,'PythonSrc') )

# imports specific to the MSD
import hdf5_getters as GETTERS

import pandas as pd

import json

AssertionError: wrong path

In [2]:
import tables

In [3]:
# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

# we define this very useful function to iterate the files
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

# we can now easily count the number of files in the dataset
print('number of song files:',apply_to_all_files(msd_subset_data_path))

# let's now get all artist names in a set(). One nice property:
# if we enter many times the same artist, only one will be kept.
all_artist_names = set()

# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()
    
# let's apply the previous function to all files
# we'll also measure how long it takes
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_artist_name)
t2 = time.time()
print('all artist names extracted in:',strtimedelta(t1,t2))




# let's see some of the content of 'all_artist_names'
print('found',len(all_artist_names),'unique artist names')
for k in range(5):
    print(list(all_artist_names)[k])

# this is too long, and the work of listing artist names has already
# been done. Let's redo the same task using an SQLite database.
# We connect to the provided database: track_metadata.db
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
# we build the SQL query
q = "SELECT DISTINCT artist_name FROM songs"
# we query the database
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print('all artist names extracted (SQLite) in:',strtimedelta(t1,t2))
# we close the connection to the database
conn.close()
# let's see some of the content
for k in range(5):
    print(all_artist_names_sqlite[k][0])

# now, let's find the artist that has the most songs in the dataset
# what we want to work with is artist ID, not artist names. Some artists
# have many names, usually because the song is "featuring someone else"
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id FROM songs"
res = conn.execute(q)
all_artist_ids = map(lambda x: x[0], res.fetchall())
conn.close()

# The Echo Nest artist id look like:
for k in range(4):
    print(all_artist_ids[k])

# let's count the songs from each of these artists.
# We will do it first by iterating over the dataset.
# we prepare a dictionary to count files
files_per_artist = {}
for aid in all_artist_ids:
    files_per_artist[aid] = 0

# we prepare the function to check artist id in each file
def func_to_count_artist_id(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_id = GETTERS.get_artist_id(h5)
    files_per_artist[artist_id] += 1
    h5.close()

# we apply this function to all files
apply_to_all_files(msd_subset_data_path,func=func_to_count_artist_id)

# the most popular artist (with the most songs) is:
most_pop_aid = sorted(files_per_artist,
                      key=files_per_artist.__getitem__,
                      reverse=True)[0]
print(most_pop_aid,'has',files_per_artist[most_pop_aid],'songs.')

# of course, it is more fun to have the name(s) of this artist
# let's get it using SQLite
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_name FROM songs"
q += " WHERE artist_id='"+most_pop_aid+"'"
res = conn.execute(q)
pop_artist_names = map(lambda x: x[0], res.fetchall())
conn.close()
print('SQL query:',q)
print('name(s) of the most popular artist:',pop_artist_names)

# let's redo all this work in SQLite in a few seconds
t1 = time.time()
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id,artist_name,Count(track_id) FROM songs"
q += " GROUP BY artist_id"
res = conn.execute(q)
pop_artists = res.fetchall()
conn.close()
t2 = time.time()
print('found most popular artist in',strtimedelta(t1,t2))
print(sorted(pop_artists,key=lambda x:x[2],reverse=True)[0])



('number of song files:', 10000)
('all artist names extracted in:', '0:01:10.435715')
('found', 4412, 'unique artist names')
Pale Forest
The Real Kids
JennyAnyKind
Little Willie John
Barry Goldberg
('all artist names extracted (SQLite) in:', '0:00:00.020937')
!!!
(hed) p.e.
089 Clique feat. Minnesota Snipe & Skinny Cueball
089 Clique feat. Prophet
1. Futurologischer Congress
AR009211187B989185
AR00A6H1187FB5402A
AR00LNI1187FB444A5
AR00MBZ1187B9B5DB1
(u'AROIHOI122988FEB8E', 'has', 13, 'songs.')
('SQL query:', u"SELECT DISTINCT artist_name FROM songs WHERE artist_id='AROIHOI122988FEB8E'")
('name(s) of the most popular artist:', [u'Mario Rosenstock'])
('found most popular artist in', '0:00:00.078373')
(u'AROIHOI122988FEB8E', u'Mario Rosenstock', 13)


In [11]:
filename = msd_subset_data_path + '/A/A/A/TRAAAPK128E0786D96.h5'
print(filename)
f = GETTERS.open_h5_file_read(filename)
print(GETTERS.get_segments_timbre(f).shape, GETTERS.get_artist_name(f))
a = GETTERS.get_segments_timbre(f)
np.savetxt("mfcc.csv", a, delimiter=",")

f.close()


./data/MSD/MillionSongSubset/data/A/A/A/TRAAAPK128E0786D96.h5
((588, 12), 'Tweeterfriendly Music')


In [12]:
f = open("track_ids.txt", "a")

def func_to_get_song_id(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    track_id = GETTERS.get_track_id(h5)
    f.write(track_id+"\n")
    h5.close()
    
apply_to_all_files(msd_subset_data_path, func=func_to_get_song_id)
f.close()

In [43]:
l = np.array([])
track_ids = []
def func_to_get_timbre(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    a = GETTERS.get_segments_timbre(h5)
    track_id = GETTERS.get_track_id(h5)
    global l
    global track_ids
    if(a.shape[0] > 300):
        l = np.concatenate((l, np.array([a[:300]]))) if l.size else np.array([a[:300]])
        track_ids += [track_id]
    h5.close()
    
apply_to_all_files(msd_subset_data_path, func=func_to_get_timbre)
print(l.shape)
print(len(track_ids))

KeyboardInterrupt: 

In [45]:
print(l.shape)
print(len(track_ids))

(7457, 300, 12)
7457


In [50]:
np.save(open("mfcc.npy", "w"), l)

In [7]:
f = open("chroma.npy", "r")
new_l = np.load(f)
print(new_l.shape)
f.close()

(7457, 300, 12)


In [53]:
f = open("track_ids_for_chroma.txt", "a")
for track in track_ids:
    f.write(track+"\n")
f.close()

In [27]:
song_dict = {}
track_dict = {}
def func_to_match_song_and_track(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    song_id = GETTERS.get_song_id(h5)
    track_id = GETTERS.get_track_id(h5)
    global song_dict
    song_dict[song_id] = track_id
    track_dict[track_id] = song_id
    h5.close()

In [28]:
apply_to_all_files(msd_subset_data_path, func=func_to_match_song_and_track)

10000

In [18]:
song_dict
df = pd.DataFrame(song_dict.items())

In [19]:
df

,0,1
0,SOKHEQY12AC468B3F9,TRBGEHK12903CEEFC0
1,SONYSNO12AB0189762,TRADAFY12903CC7EAA
2,SOMWKPQ12A679D8AEA,TRBHGWP128E0793AD8
3,SOSKUNI12AB0187F12,TRAASQC128F93480F0
4,SOEDGSG12AB0184A57,TRADFBR128F9308456
5,SOEYVTH12A8C138E6E,TRAQKIM128F42618D0
6,SOXBEKP12A6D4F979E,TRAZOSO128F149739D
7,SOMRGMY12A6D4F93A8,TRAFZHK128F145E86B
8,SOBHMQL12A67ADE30A,TRALDWN128EF33FB6E
9,SONTKHG12A81C22D4C,TRBBANU128F422B87B


In [22]:
with open('data/song-track-mapping.json', 'w') as fp:
    json.dump(song_dict, fp)

In [26]:
with open('data/song-track-mapping.json', 'rb') as fp:
    new_song_dict = json.load(fp)
new_song_dict


{u'SOKHEQY12AC468B3F9': u'TRBGEHK12903CEEFC0',
 u'SONYSNO12AB0189762': u'TRADAFY12903CC7EAA',
 u'SOMWKPQ12A679D8AEA': u'TRBHGWP128E0793AD8',
 u'SOSKUNI12AB0187F12': u'TRAASQC128F93480F0',
 u'SOEDGSG12AB0184A57': u'TRADFBR128F9308456',
 u'SOEYVTH12A8C138E6E': u'TRAQKIM128F42618D0',
 u'SOXBEKP12A6D4F979E': u'TRAZOSO128F149739D',
 u'SOMRGMY12A6D4F93A8': u'TRAFZHK128F145E86B',
 u'SOKJDLN12AB01826D9': u'TRATARQ128F932BD75',
 u'SOPPTXH12A8C1409BC': u'TRBEELS128F42B63A8',
 u'SOQYNGM12A6D4F89D8': u'TRARVMJ128F146A77E',
 u'SOKKHGP12A8C13768C': u'TRALSVZ128F4264341',
 u'SOQVHOW12AC46885EE': u'TRAQFUM12903D0F30B',
 u'SOMFZZQ12A8C13AB56': u'TRAFGKU128F42889EF',
 u'SOJVZZA12A58A79CC3': u'TRAOGLL128F931FAEB',
 u'SOVICTQ12A8C137F0D': u'TRBAHPZ128F427FCEA',
 u'SOBDGBW12A8C13D50B': u'TRBHOBW128F429504D',
 u'SOOJKNX12A8C132425': u'TRADSQV128F425F3D2',
 u'SOGAEXV12A8C138122': u'TRAVTMG128F4262F39',
 u'SOJQTVV12A8C134387': u'TRBCORX128F4275BEB',
 u'SOZVJGT12A8C137370': u'TRAHLVW128F426684F',
 u'SOFAJOM12A

In [29]:
with open('data/track-song-mapping.json', 'w') as fp:
    json.dump(track_dict, fp)

In [25]:
#Explore genre data:
genre_dict = {}
genre_set = set()
with open('data/MSD/msd_tagtraum_cd2.cls', 'r') as fp:
    lines = fp.readlines()
    for i in range(7, len(lines)):
        split = lines[i].strip().split()
        genre_set.add(split[1])
        genre_dict[split[0]] = split[1]

In [33]:
print(genre_dict['TRAACPE128F421C1B9'])
with open('data/track-genre-mapping.json', 'w') as fp:
    json.dump(genre_dict, fp)

RnB


In [29]:
print(len(genre_set))
genre_set


15


{'Blues',
 'Country',
 'Electronic',
 'Folk',
 'Jazz',
 'Latin',
 'Metal',
 'New',
 'Pop',
 'Punk',
 'Rap',
 'Reggae',
 'RnB',
 'Rock',
 'World'}

In [5]:

l = np.array([])
track_ids = []
def func_to_get_chroma(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    a = GETTERS.get_segments_pitches(h5)
    track_id = GETTERS.get_track_id(h5)
    global l
    global track_ids
    if(a.shape[0] > 300):
        l = np.concatenate((l, np.array([a[:300]]))) if l.size else np.array([a[:300]])
        track_ids += [track_id]
    h5.close()
    
apply_to_all_files(msd_subset_data_path, func=func_to_get_chroma)
print(l.shape)
print(len(track_ids))

(9484, 300, 12)
9484


In [6]:
np.save(open("data/chroma.npy", "w"), l)